In [2]:
import requests
import json
from tqdm import tqdm
import time

예시코드 listing_id = '6261834'

In [2]:
url = "https://www.airbnb.co.kr/api/v2/reviews"
params = {
    'key':'d306zoyjsyarp7ifhu67rjxn52tv0t20',
    'currency':'KRW',
    'locale':'ko',
    'listing_id':'6261834',
    'role':'guest',
    '_format':'for_p3',
    '_limit':'100', # 몇개의 게시물을 보여줄 것인가(100개 이하로 설정가능)
    '_offset':'0',# 몇번째 게시물부터 시작할 것인가 0부터 시작
    '_order':'language_country'}

##### https://www.airbnb.co.kr/api/v2/reviews?key=d306zoyjsyarp7ifhu67rjxn52tv0t20&currency=KRW&locale=ko&listing_id=6261834&role=guest&_format=for_p3&_limit=7&_offset=7&_order=language_country

In [3]:
response = requests.get(url,params=params).text

In [4]:
result = json.loads(response)

- result:{'reviews', 'metadata'}
    -  reviews:[100 items]: {'collection_tag', __'comments'__, 'created_at', __'id'__, 'language', 'localized_date', 'rating', 'response', 'reviewee', __'reviewer'__}
    - metadata:{'reviews_count', 'should_show_review_translations'}

In [5]:
for k,v in result['reviews'][0].items():
     print("|| %30s || : %s" % (k,v)) # (|| key || : value) 형태로 출력

||                 collection_tag || : None
||                       comments || : 너무 깨끗하고 좋았습니다.
||                     created_at || : 2018-10-29T07:04:45Z
||                             id || : 342585456
||                       language || : ko
||                 localized_date || : 2018년 10월
||                         rating || : 5
||                       response || : 
||                       reviewee || : {'deleted': False, 'first_name': '현아', 'host_name': '현아', 'id': 24468677, 'is_superhost': False, 'picture_url': 'https://a0.muscache.com/im/pictures/5af5aaac-5719-4708-8056-b32d76f23b44.jpg?aki_policy=profile_x_medium', 'profile_path': '/users/show/24468677'}
||                       reviewer || : {'deleted': False, 'first_name': 'Doohoon', 'host_name': 'Doohoon', 'id': 81515153, 'is_superhost': False, 'picture_url': 'https://a0.muscache.com/im/pictures/user/c9774fbd-6469-4db7-81a1-f374743fcad8.jpg?aki_policy=profile_x_medium', 'profile_path': '/users/show/81515153'}


### listing_id 이용한 리뷰데이터 크롤링(서울)

In [6]:
import requests
import json
from tqdm import tqdm
import time

In [7]:
# 14610개 숙소 정보담은 home_jsn_314610,json 파일 불러오자
with open('home_jsn_14610.json','r') as fp: 
    home_jsn = json.load(fp)

14610개 숙소의 모든 리뷰를 긁어오자

In [8]:
listing_ids = list(home_jsn.keys())

In [9]:
url = "https://www.airbnb.co.kr/api/v2/reviews"
headers = {'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}
# header에 User Agent를 붙여서 bot이 아님을 보여줌

params = {
        'key':'d306zoyjsyarp7ifhu67rjxn52tv0t20',
        'currency':'KRW',
        'locale':'ko',
#         'listing_id':
        'role':'guest',
        '_format':'for_p3',
        '_limit':'100', # 몇개의 게시물을 보여줄 것인가(100개 이하로 설정가능)
#         '_offset':'0',# 몇번째 게시물부터 시작할 것인가 0부터 시작
        '_order':'language_country'
            }

review_jsn ={} # 리뷰데이터를 json으로 저장하기위해 우선 dict 생성
review_cnt_total = 0

for listing_id in tqdm(listing_ids): # 숙소id listing_id 대해
    review_jsn[listing_id] = [] # listing_id 키로 갖는 리뷰리스트 쌓자
    
    params['listing_id'] = listing_id # 리스팅 id를 해당 listing_id로 설정
    
    retry_cnt = 2 # 최대 2번동안 listing_id에 해당하는 response 요청하겠다 
    while retry_cnt !=0:
        try:
            listing_id_res = requests.get(url, params=params, headers=headers).json()
            break
        except Exception as e: # response 못받은 경우 retry
            print('retry in listing_id!!',e)
            retry_cnt -= 1
            time.sleep(2)
            continue
            
    review_count= listing_id_res['metadata']['reviews_count']
    review_cnt_total += review_count
#     print('review_cnt_total: ',review_cnt_total)
    
    review_count_iter = review_count//100 # 100개단위로 리뷰개수 끊어줌
    review_count_rest = review_count%100 # 100개 미만 나머지 리뷰의 수
    
    for i in range(review_count_iter): # 리뷰개수 100개 단위로 반복
        params['_offset'] = 100*i
        retry_cnt = 2
        
        while retry_cnt !=0: # 최대 2번 동안 request
            try:
                res_100_reviews = requests.get(url, params=params, headers=headers).json()
                break
            except Exception as e: # response 못받은 경우 retry
                print('retry in review!!',e)
                retry_cnt -= 1
                time.sleep(2)
                continue
        
     
        review_jsn[listing_id].extend(res_100_reviews['reviews']) # listing_id 를 key로 100개의 리뷰데이터를 쌓음
        
            
    params['limit'] = review_count_rest # 100개 미만 나머지 리뷰에 대해
    while retry_cnt !=0: # 최대 2번 동안 request
            try:
                res_rest_reviews = requests.get(url, params=params, headers=headers).json()
                break
            except Exception as e: # response 못받은 경우 retry
                print('retry in review!!',e)
                retry_cnt -= 1
                time.sleep(2)
                continue
                
    review_jsn[listing_id].extend(res_rest_reviews['reviews'])

  0%|          | 1/14610 [00:02<10:32:22,  2.60s/it]


KeyboardInterrupt: 

In [34]:
print(review_cnt_total)

147323


결과를 json 파일로 저장하자

In [10]:
with open('review_seoul.json','w') as fp:
    json.dump(review_jsn,fp,ensure_ascii=False)

불러오자

- review_dict: {listing_id: [review_lst]} 형태이다

In [11]:
with open('review_seoul.json','r') as fp:
    review_dict = json.load(fp)

In [13]:
review_dict['18551259'][0]

{'collection_tag': None,
 'comments': '새벽이 도착했음에도 편안히 잘 쉴수 있었던 공간이었습니다. 반갑게 맞아준 헨리도 기억에 남네요. 너무 늦게 들어가서 놀아줄 시간이 없어 아쉬웠어요^^',
 'created_at': '2018-10-06T06:15:10Z',
 'id': 332696014,
 'language': 'ko',
 'localized_date': '2018년 10월',
 'rating': 5,
 'response': '',
 'reviewee': {'deleted': False,
  'first_name': 'David',
  'host_name': 'David',
  'id': 76010772,
  'is_superhost': False,
  'picture_url': 'https://a0.muscache.com/im/pictures/user/09c053d6-07a4-4c0e-a864-7fce8458baf6.jpg?aki_policy=profile_x_medium',
  'profile_path': '/users/show/76010772'},
 'reviewer': {'deleted': False,
  'first_name': 'Junghee',
  'host_name': 'Junghee',
  'id': 111507566,
  'is_superhost': False,
  'picture_url': 'https://a0.muscache.com/im/pictures/c1936b91-a6f7-4702-9db6-48df6ce827f9.jpg?aki_policy=profile_x_medium',
  'profile_path': '/users/show/111507566'}}